In [ ]:
! pip install sentence-transformers

In [ ]:
#import clickhouse_connect
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import math
from collections import Counter
from itertools import groupby
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
clean_comments = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/clean_comments.csv")
clean_comments = clean_comments.drop("Unnamed: 0", axis=1)

In [ ]:
clean_comments

,channel_name,comment_text,likes
0,kellyclarksonVEVO,things that made me stronger child hood versio...,4
1,kellyclarksonVEVO,damn it s been 10 years but the song is still ...,0
2,kellyclarksonVEVO,december 2019,2
3,kellyclarksonVEVO,i love music stronger,1
4,kellyclarksonVEVO,hope you didn t stack up on it during the nigh...,0
...,...,...,...
10527744,星野源 Gen Hoshino,man thanks to weathering with you using this s...,6
10527745,星野源 Gen Hoshino,i don t understand the song but still i love it,3
10527746,星野源 Gen Hoshino,i was in mission searching ken hirai song and ...,1
10527747,星野源 Gen Hoshino,아 노래 조타 i japan in korea,4


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('intfloat/e5-small',device='cuda')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import numpy as np

# tạo mảng rỗng 0 x 384
empty_embeddings = np.empty((0, 384), dtype=np.float32)

# lưu vào file .npy
np.save("/content/drive/MyDrive/Colab Notebooks/embeddings.npy", empty_embeddings)

with open("/content/drive/MyDrive/Colab Notebooks/last_index.txt", 'w') as f:
    f.write("0")


In [ ]:
def embedding_batch(batch_size, saved_path, checkpoint_path):
  with open(checkpoint_path, 'r') as f:
    index = int(f.read())
    print(f"Start with index {index}")
    old_embeddings = np.load(saved_path, allow_pickle=True)
    print(f"Old embeddings shape is {old_embeddings.shape}")
    batch = clean_comments.iloc[index:index+batch_size]['comment_text']
    batch = batch.tolist()
    embeddings = model.encode(batch, show_progress_bar=True)
    embeddings = np.concatenate((old_embeddings, embeddings))
    with open(checkpoint_path, 'w') as f:
      f.write(str(index+batch_size))
    np.save(saved_path, embeddings)
    return index+batch_size


In [ ]:
embedding_batch(batch_size,saved_path, checkpoint_path) # đừng chạy cái này

Start with index 10010000
Old embeddings shape is (0, 384)


Batches:   0%|          | 0/16180 [00:00<?, ?it/s]

11010000

In [ ]:
#embedding_batch(batch_size, saved_path, checkpoint_path)
i = 0
batch_size = 1000000
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/last_index.txt"
running = True
while running:
  try:
    saved_path = f"/content/drive/MyDrive/Colab Notebooks/embeddings{i}.npy"
    # tạo mảng rỗng 0 x 384
    empty_embeddings = np.empty((0, 384), dtype=np.float32)

    # lưu vào file .npy
    np.save(saved_path, empty_embeddings)
    embedding_batch(batch_size,saved_path, checkpoint_path)
  except:
    running = False
    print("Done")

In [ ]:
embeddings = np.load("/content/drive/MyDrive/Colab Notebooks/embeddings10.npy", allow_pickle = True) # Này để kiểm tra nhé
embeddings.shape

(517749, 384)

In [ ]:
np.save("/content/drive/MyDrive/Colab Notebooks/embeddings11.npy", embeddings)